# To train and test Yolov5, the first clone the repository. All dependencies must be installed an imported as well

In [ ]:
!git clone https://github.com/achelm15/yolov5
%cd yolov5
!pip install -r requirements.txt
import pathlib
import os
import cv2
import zipfile
from os import path
import shutil
import csv
import random
import numpy as np
from tensorflow import keras
import tensorflow_datasets as tfds
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import tensorflow as tf
import numpy as np
import glob
from shutil import copyfile
from PIL import Image
from math import floor, ceil

Cloning into 'yolov5'...
remote: Enumerating objects: 10893, done.
remote: Total 10893 (delta 0), reused 0 (delta 0), pack-reused 10893
Receiving objects: 100% (10893/10893), 12.39 MiB | 26.43 MiB/s, done.
Resolving deltas: 100% (7532/7532), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# The next cells contain necessary steps to train the model on the Kitti dataset

In [ ]:
#creates a yaml file to train YOLOv5 on the Kitti dataset
data = open("data.yaml", "w")
data.write("train: ./kittiyolo/train/images\nval: ./kittiyolo/valid/images\nnc: 8\nnames: [\'Car\', \'Pedestrian\', \'Cyclist\', \'Van\', \'Truck\', \'Person_sitting\', \'Tram\', \'DontCare\']")
data.close()

In [ ]:
#downloads the kitti images in a zip file
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip -O kitti.zip

--2022-04-07 18:23:44--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.171.17
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.171.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12569945557 (12G) [application/zip]
Saving to: ‘kitti.zip’

kitti.zip           100%[===================>]  11.71G  30.3MB/s    in 6m 59s  

2022-04-07 18:30:45 (28.6 MB/s) - ‘kitti.zip’ saved [12569945557/12569945557]



In [ ]:
#downloads the kitti labels in a zip file
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip -O kittilabel.zip

--2022-04-07 18:30:45--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.169.81
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.169.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5601213 (5.3M) [application/zip]
Saving to: ‘kittilabel.zip’

kittilabel.zip      100%[===================>]   5.34M  6.70MB/s    in 0.8s    

2022-04-07 18:30:46 (6.70 MB/s) - ‘kittilabel.zip’ saved [5601213/5601213]



In [ ]:
# unzips both the images and labels
with zipfile.ZipFile("./kitti.zip", 'r') as zip_ref:
    zip_ref.extractall("kitti")

import zipfile
with zipfile.ZipFile("./kittilabel.zip", 'r') as zip_ref:
    zip_ref.extractall("kittilabel")

The next few boxes convert the original labels of the Kitti dataset to a form that YOLOv5 is able to use. If you wish to understand this transformation, open a Kitti label file before running the next few cells, and then open the corresponding file after running the cells.

In [ ]:
data_dir = pathlib.Path('kitti/training/image_2')
label_dir = pathlib.Path('kitti/training/label_2')
yolo_labels = pathlib.Path('yolo')

In [ ]:
image_count = len(list(data_dir.glob('*.png')))
label_count = len(list(label_dir.glob('*.txt')))

In [ ]:
len(list(pathlib.Path('./kittilabel/training/label_2').glob('*txt')))

7481

In [ ]:
root_path = "./kitti/training/"
yolo_path = "yolo/"

In [ ]:
kitti2yolotype_dict = {
    "Car": "0",
    "Van": "0",
    "Pedestrian": "1",
    "Person_sitting": "1",
    "Cyclist": "2",
    "Truck": "3",
    "Tram": "6",
    "Misc": "6",
    "DontCare": "6",
}
def kitti2yolo(kitti_label, img_height, img_width):

    kitti_label_arr = kitti_label.split(" ")
    x1 = float(kitti_label_arr[4])
    y1 = float(kitti_label_arr[5])
    x2 = float(kitti_label_arr[6])
    y2 = float(kitti_label_arr[7])

    bb_width = x2 - x1
    bb_height = y2 - y1
    yolo_x = (x1 + 0.5 * bb_width) / img_width
    yolo_y = (y1 + 0.5 * bb_height) / img_height
    yolo_bb_width = bb_width / img_width
    yolo_bb_height = bb_height / img_height
    yolo_label = kitti2yolotype_dict[kitti_label_arr[0]]

    return (
        yolo_label
        + " "
        + str(yolo_x)
        + " "
        + str(yolo_y)
        + " "
        + str(yolo_bb_width)
        + " "
        + str(yolo_bb_height)
    )

In [ ]:
%cp -r kittilabel/training/label_2 kitti/training/label_2

In [ ]:
kitti_images_path = root_path + "image_2" + os.sep
kitti_labels_path = root_path + "label_2" + os.sep

# yolo paths
if yolo_path is None:
    yolo_path = root_path + "yolo_labels" + os.sep

if not os.path.exists(yolo_path):
    os.makedirs(yolo_path)

# load each kitti label, convert to yolo and save
for labelfilename in os.listdir(kitti_labels_path):
  yolo_labels = []
  with open(kitti_labels_path + labelfilename, "r") as kittilabelfile:
    cvimage = cv2.imread(
        kitti_images_path + labelfilename.split(".txt")[0] + ".png"
        )
    height, width, frame_depth = cvimage.shape
    for kitti_label in kittilabelfile:
      yolo_labels.append(
          kitti2yolo(kitti_label, img_height=height, img_width=width)
          )
      with open(yolo_path + labelfilename, "w+") as yololabelfile:
        for label in yolo_labels:
          yololabelfile.write(label + "\n")

The next cells move the correct YOLOv5 labels to a directory so they can be used to train the model

In [ ]:
%mkdir kittiyolo
%cd kittiyolo
%mkdir train
%mkdir valid
%cd valid
%mkdir images
%mkdir labels
%cd ../..

/content/yolov5/kittiyolo
/content/yolov5/kittiyolo/valid
/content/yolov5


In [ ]:
%cp -r yolo kittiyolo/train/labels
%cp -r kitti/training/image_2 kittiyolo/train/images

In [ ]:
root_path = "./kittiyolo/train/"
kittiyolo_images_path = root_path + "images" + os.sep
kittiyolo_labels_path = root_path + "labels" + os.sep
yolo_valid_path = "./yolo/valid"

# load each kitti label, convert to yolo and save
count = 0
for labelfilename in os.listdir(kittiyolo_images_path):
  count = count+1
  if count == 3:
    break

In [ ]:
def moveFiles(src, dst):
  files = []
  for i in os.listdir(src):
    files.append(i)
  files = sorted(files)
  listt = np.random.RandomState(seed=42).permutation(files)[:1000]
  for f in listt:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

In [ ]:
moveFiles("kittiyolo/train/images", "kittiyolo/valid/images")
moveFiles("kittiyolo/train/labels", "kittiyolo/valid/labels")

#Used to augment the images

In [ ]:
%cp -r kittiyolo kittiyolosave

In [ ]:
%rm -r kittiyolo
%cp -r kittiyolosave kittiyolo

In [ ]:
%ls kittiyolo/train/images | wc -l
%ls kittiyolo/train/labels | wc -l
%ls kittiyolo/valid/images | wc -l
%ls kittiyolo/valid/labels | wc -l

6481
6481
1000
1000


In [ ]:
def contrast_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_contrast(img, 2, 5, 42)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "contrast", folder)
def noise_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.stateless_random_jpeg_quality(img, 2, 20, (42,42))
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "noise", folder)
def grayscale_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.rgb_to_grayscale(img)
    img = img.numpy()
    img = img.reshape((img.shape[0], img.shape[1]))
    im = Image.fromarray(img, 'L')
    im.save(folder+'/images/'+new_image)
    copy_label(image, "grayscale", folder)
def saturate_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_saturation(img, 2, 120, 42)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "saturate", folder)
def hue_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_hue(img, 0.1)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "hue", folder)
def flip_image_lr(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.flip_left_right(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    flip_label_lr(image[:image.find('.')]+".txt", image[:image.find('.')]+"_flip_lr.txt", folder)
def flip_image_ud(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.flip_up_down(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    flip_label_ud(image[:image.find('.')]+".txt", image[:image.find('.')]+"_flip_ud.txt", folder)
def rot_90(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.rot90(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    label_rot_90(image[:image.find('.')]+".txt", image[:image.find('.')]+"_rot_90.txt", folder)
def crop_bbox(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    width = img.shape[1]
    height = img.shape[0]
    label_file = folder+"/labels/"+image[:image.find('.')]+".txt"
    ball_list = []
    with open(label_file) as f:
        count = 0
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            if words[0]=="1":
                ball_list.append(words)
                xp = float(words[1])
                yp = float(words[2])
                wp = float(words[3])
                hp = float(words[4])
                x1 = int(floor(xp*width-(wp*width)/2))
                y1 = int(floor(yp*height-(hp*height)/2))
                image = tf.image.crop_to_bounding_box(img,y1,x1,int(ceil(hp*height)),int(ceil(wp*width)))
                image = image.numpy()
                im = Image.fromarray(image)
                file = new_image[0:new_image.find('.')]+"_"+str(count)+new_image[new_image.find('.'):]
                im.save(folder+'/images/'+file)
                new_label_file = file[:file.find('.')]+".txt"
                bb_crop_label(new_label_file, folder)
                count += 1
def bb_crop_label(new_label_file, folder):
    with open(folder+'/labels/'+new_label_file, 'w') as f:
        f.write("1 0.5 0.5 1.0 1.0")
def flip_label_lr(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            words[1] = str(1-float(words[1]))
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def flip_label_ud(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            words[2] = str(1-float(words[2]))
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def label_rot_90(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            x = words[1]
            y = words[2]
            w = words[3]
            h = words[4]
            words[1] = y
            words[2] = str(1-float(x))
            words[3] = h
            words[4] = w
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def copy_label(image, augmentation, folder):
    filename = image[:image.find('.')]
    new_filename = folder+'/labels/'+filename+'_'+augmentation+'.txt'
    filename = folder+'/labels/'+filename+'.txt'
    copyfile(filename, new_filename)

In [ ]:
# aug_list = ['_cropped', '_contrast', '_noise', '_flip_lr', '_flip_ud', '_rot_90', '_saturate', '_hue']
# aug_list = ['_contrast', '_noise' '_saturate']
aug_list = ['_noise']
data_dir = "./kittiyolo/train"

In [ ]:
for root, dirs, files in os.walk(data_dir+'/images/', topdown=False):
  for file in files:
    count = 0
    for j in aug_list:
      filename = file[:file.find('.')]+j+file[file.find('.'):]
      # if count==0: contrast_image(file, filename, data_dir)
      # if count==1: noise_image(file, filename, data_dir)
      # if count==2: saturate_image(file, filename, data_dir)
      noise_image(file, filename, data_dir)
      count+=1

In [ ]:
for root, dirs, files in os.walk(data_dir+'/images/', topdown=False):
    for file in files:
        filename = file[:file.find('.')]+"_saturate"+file[file.find('.'):]
        saturate_image(file, filename, data_dir)

In [ ]:
%cp -r kittiyolorestart kittiyolo/

# The next cell can be used to train the dataset on the small version of YOLOv5. 
Changing the parameters might lead to improved performance. See the detect.py file for information on parameters

In [ ]:
!python train.py --img 320 --batch 175 --epochs 350 --data ./data.yaml --weights "yolov5s.pt" --project modified --freeze 1 3 5 7 9 --changes 2,4,6,8,13 --hyp data/hyps/hyp.finetune.yaml 

In [ ]:
!python val.py --weights "modified/train/exp/best.pt" --img 320 --data data.yaml